# Titanic: Predict survival on the Titanic with AWS Glue.
### 타이타닉 사고에서 어떤 승객이 살아남을 수 있을 지 예측해보세요
##### Kaggle은 2010년 설립된 예측모델 및 분석 대회 플랫폼입니다.
##### 기업 및 단체에서 데이터와 해결과제를 등록하면, 데이터 과학자들이 이를 해결하는 모델을 개발하고 경쟁하는 곳입니다. 
##### 이번 Lab에서는 Kaggle에서 입문자용 tutorial로 사용되는 titanic competition을 Glue와 Spark ML을 사용해서 데이터 분석, ETL, ML을 사용한 Prediction까지 실습해보도록 하겠습니다.

![titanic_sinking](images/titanic_sinking.jpg)

#### 필요한 라이브러리 Import

In [ ]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean, col, split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

### 1. Glue 카탈로그에서 필요한 데이터 준비

In [ ]:
from awsglue.context import GlueContext
 
glueContext = GlueContext(sc)
titanic_df = glueContext.create_dynamic_frame.from_catalog(database='analytics_hol',
                                                           table_name='titanic_train',                           
                                                           transformation_ctx='titanic_df').toDF()
titanic_df.show()

##### passengerid: 승객 id
##### survived: 생존여부
##### pclass: 티켓등급 (1=1st, 2=2nd, 3=3rd)
##### name: 이름
##### sex: 성별 (0=male, 1=female)
##### age: 나이
##### sibsp: 함께 탑승한 형제자매, 배우자의 수
##### parch: 함께 탑승한 부모, 자식의 수
##### ticket: 티켓번호
##### fare: 운임
##### cabin: 객실 번호
##### embarked: 탑승항구 (C=Cherbourg, Q=Queenstown, S=Southampton)

#### 1.1. 전체 데이터 수 확인

In [ ]:
titanic_df.count()

#### 1.2. 데이터 스키마 확인

In [ ]:
titanic_df.printSchema()

#### 1.3. 사용할 데이터에 모든 컬럼에 대해 요약 통계를 확인합니다.

In [ ]:
titanic_df.describe().show()

##### 전체 데이터가 891개로 다른 모든 컬럼은 891개의 데이터를 모두 가지고 있지만 age만 714개로 누락된 데이터가 존재함을 확인합니다.
##### 빈 스트링은 이 요약 통계에서 구분이 불가능함으로 이후에 별도의 확인 과정을 통해 따로 체크하도록 합니다.

### 2. Exploratory data analysis

#### 2.1. 탑승객 생존율 확인

In [ ]:
titanic_df.groupBy('survived').count().show()

##### 891명의 승객 중 342명만 생존했음을 알 수 있습니다.
##### 생존율은 sex, age, pclass와 같은 승객 특징들과 관련이 있으므로 해당 feature에 따른 생존율을 살펴봅니다.

#### 2.2. 성별에 따른 생존율 확인

In [ ]:
titanic_df.groupBy('sex', 'survived').count().orderBy('sex', 'survived', ascending=[1, 1]).show()

##### 남성보다 여성의 생존율이 높음을 알 수 있습니다.

#### 2.3. 티켓등급에 따른 생존율 확인

In [ ]:
titanic_df.groupBy('pclass', 'survived').count().orderBy('pclass', 'survived', ascending=[1, 1]).show()

##### 1등석의 생존율은 높고, 3등석의 생존율은 낮은 것을 알 수 있습니다.

#### 2.4. 함께 탑승한 형제자매나 배우자에 따른 생존율 확인

In [ ]:
titanic_df.groupBy('sibsp', 'survived').count().orderBy('sibsp', 'survived', ascending=[1, 1]).show()

##### 동승한 형제자매나 배우자가 없는 경우 생존율이 낮은 것을 알 수 있습니다.

#### 2.5. 함께 탑승한 부모님이나 자녀에 따른 생존율 확인

In [ ]:
titanic_df.groupBy('parch', 'survived').count().orderBy('parch', 'survived', ascending=[1, 1]).show()

##### 동승한 부모나 자녀가 없는 경우 생존율이 낮은 것을 알 수 있습니다.

#### 2.6 탑승항구에 따른 생존율 확인

In [ ]:
titanic_df.groupBy('embarked', 'survived').count().orderBy('embarked', 'survived', ascending=[1, 1]).show()

##### 2개의 데이터는 탑승항구가 명시되어 있지 않는 것을 알 수 있습니다.
##### 탑승항구 S(Southampton)의 생존율이 낮은 것을 알 수 있습니다.
##### 따라서, 탑승항구 S(Southampton)의 생존율이 낮은 이유를 알기 위해 탑승항구와 탑승석의 관계를 살펴볼 필요가 있습니다.

In [ ]:
titanic_df.groupBy('embarked', 'pclass').count().orderBy('embarked', 'pclass', ascending=[1, 1]).show()

##### 탑승항구 S(Southampton)의 생존율이 낮은 이유는 대다수 탑승객이 3등석이기 때문임을 알 수 있습니다.

### 3. Feature engineering

#### 3.1. Null Column Check
##### Null 값이나 empty string을 가진 Column이 존재하는 지 확인합니다.

##### 3.1.1. Null Value를 체크하기 위한 UDF(User Defined Function) 함수 선언

In [ ]:
def count_null_or_empty_value(df):
    null_columns_counts = []
    
    for column in df.columns:
        null_rows = df.where((col(column).isNull()) | (col(column) == '')).count()
    
        if(null_rows > 0):
              null_columns_counts.append((column, null_rows))
    return(null_columns_counts)

##### 3.1.2. count_null_or_empty_value UDF를 이용하여 Null Value feature 확인

In [ ]:
null_columns_count_list = count_null_or_empty_value(titanic_df)
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

##### age feature가 177개의 null value를 가진 것이 확인되었습니다.
##### cabin feature가 687개의 empty string을 가진 것이 확인되었습니다.
##### embarked feature가 2개의 empty string을 가진 것이 확인되었습니다.
##### 이 정보는 이후 진행되는 feature engineering에서 참고 자료로 사용합니다.

#### 3.2. Name

In [ ]:
titanic_df.select("name").show()

##### 샘플 데이터에서 name field를 살펴보면 그대로 사용하기는 어렵고 정리가 필요하다는 것을 알 수 있습니다.
##### name field에 나이와 성별 기혼 등을 의미하는 호칭(Mr, Mrs, Miss)이 있는 것을 알 수 있습니다. 

##### 3.2.1. name column에서 유의미한 호칭만 추출하여 별도의 initial column으로 만들고 기존 데이터에 추가

In [ ]:
titanic_df = titanic_df.withColumn('initial', regexp_extract(col('name'), "(\w+)\.", 1))
titanic_df.show()

##### 3.2.2 생성된 initial feature의 데이터 분포 확인 
##### MR: 성인 남성
##### Miss: 여자 아동
##### Mrs: 기혼 여성
##### Master: 남자 아동
##### MS: 성인 여성

In [ ]:
titanic_df.groupBy('initial').count().orderBy('count', ascending=[0]).show()

##### Mlle나 Mme 같이 Miss를 다르게 표현한 호칭도 확인할 수 있고, Master까지만 유의미하고 나머지는 하나로 그룹화하여 Other로 통합하는 것이 이후 데이터 분석에 효율적일 것으로 생각됩니다.  

In [ ]:
titanic_df = titanic_df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])
titanic_df.groupBy("initial").count().orderBy("count", ascending=[0]).show()

##### 최종 정리된 Initial feature의 모습을 확인할 수 있습니다.

#### 3.3. Age

In [ ]:
titanic_df.where((col('age').isNull()) | (col('age') == '')).count()

##### age에 177개의 빈 값은 적절하게 채워줘야 나중에 모델링할 때 성능을 높일 수 있습니다.

##### 3.3.1. age의 평균값을 확인

In [ ]:
# mean('age')의 output column 이름이 mean(age)여서 mean_age로 alias 함 
titanic_df.select(mean('age').alias('mean_age')).show()

##### 3.3.2. 전체 age의 평균은 29.70이나 우리는 initial feature를 알고 각 initial age 평균을 살펴봅니다.

In [ ]:
import pyspark.sql.functions as sf

# avg('age')의 output column 이름이 avg(age)여서 avg_age로 alias 함 
titanic_df.groupby('initial').agg(sf.avg('age').alias('avg_age')).orderBy('avg_age', ascending=[1]).show()

##### 3.3.3. 각 initial age 평균을 알 수 있으므로 우리는 빈 age 값을 이를 이용하여 좀 더 스마트하게 채워줄 수 있습니다.

In [ ]:
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Miss') & (titanic_df['age'].isNull()), 22).otherwise(titanic_df['age']))
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Other') & (titanic_df['age'].isNull()), 46).otherwise(titanic_df['age']))
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Master') & (titanic_df['age'].isNull()), 5).otherwise(titanic_df['age']))
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Mr') & (titanic_df['age'].isNull()), 33).otherwise(titanic_df['age']))
titanic_df = titanic_df.withColumn('age', when((titanic_df['initial'] == 'Mrs') & (titanic_df['age'].isNull()), 36).otherwise(titanic_df['age']))

##### 3.3.4. age의 요약 통계를 다시 확인합니다.

In [ ]:
titanic_df.select(col('age')).describe().show()

##### null 값 없이 891개의 값이 존재하며, 평균도 이전 값을 유지하고 있는 것을 확인할 수 있습니다.

##### 3.3.5. initial에 따른 생존율을 확인합니다.

In [ ]:
titanic_df.groupBy('initial', 'survived').count().orderBy('initial', 'survived', ascending=[1, 1]).show()

##### 남성일수록 나이가 많을수록 생존율이 낮은 걸 확인할 수 있습니다.

#### 3.4. Embarked

In [ ]:
titanic_df.groupBy('embarked').count().show()

##### 3.4.1. 대부분의 승객이 'S'에 해당하고 missing 값은 단 2개 뿐이므로 missing 값을 'S'로 채워줘도 큰 문제는 없습니다.

In [ ]:
titanic_df = titanic_df.na.fill({'embarked' : 'S'})

#### 3.5. Cabin

In [ ]:
titanic_df.where((col('cabin').isNull()) | (col('cabin') == '')).count()

##### 3.5.1. cabin feacture는 대다수의 값이 빈 값으로 사용하기 어렵다고 판단되므로 drop 하도록 하겠습니다.

In [ ]:
titanic_df = titanic_df.drop('cabin')

In [ ]:
titanic_df.printSchema()

##### Schema에서 cabin feature가 drop 된 것을 확인할 수 있습니다.

#### 3.6. Family Size

##### 3.6.1. sibsp와 parch를 사용해서 family_size feature를 만들어 사용하도록 하겠습니다.

In [ ]:
titanic_df = titanic_df.withColumn('family_size', col('sibsp') + col('parch'))

In [ ]:
titanic_df.groupBy('family_size').count().orderBy('count', ascending=[0]).show()

##### 3.6.2. family_size를 확인하니 대다수는 0이어서 alone feature를 따로 만들어 모델링에 사용하도록 하겠습니다.
##### alone feature의 기본 값은 0으로 설정합니다.

In [ ]:
titanic_df = titanic_df.withColumn('alone', lit(0))

##### family_size가 0이면 alone feature를 1로 설정합니다.

In [ ]:
titanic_df = titanic_df.withColumn('alone', when(titanic_df['family_size'] == 0, 1).otherwise(titanic_df['alone']))

#### 3.7. Create Index features

##### 3.7.1. 현재까지 생성된 column들을 살펴봅니다.

In [ ]:
titanic_df.columns

##### 3.7.2. 이 중 sex, embarked, initial feature는 값이 string으로 되어있어 모델링에 사용할 수 없습니다. StringIndexer를 이용해서 number로 변환한 후 별도의 column으로 추가합니다.

In [ ]:
indexers = [StringIndexer(inputCol=column, outputCol=column+'_index').fit(titanic_df) for column in ['sex', 'embarked', 'initial']]
pipeline = Pipeline(stages=indexers)
titanic_df = pipeline.fit(titanic_df).transform(titanic_df)
titanic_df.show()

##### 3.7.3. 변경된 Schema를 확인합니다.

In [ ]:
titanic_df.printSchema()

#### 3.8. Drop Columns

##### 3.8.1. 모델링에서 사용하지 않는 column들은 Drop 하도록 합니다.

In [ ]:
titanic_df = titanic_df.drop('passengerid', 'name', 'ticket', 'cabin', 'embarked', 'sex', 'initial')

##### 모델링에 사용할 최종 데이터를 확인합니다.

In [ ]:
titanic_df.show()

#### 3.9. Put all features into vector

##### 3.9.1. 모델에서 해당 feature들을 사용하기 위해서 VectorAssembler를 이용해서 vector화 합니다.

In [ ]:
feature = VectorAssembler(inputCols=titanic_df.columns[1:], outputCol='features')
feature_vector= feature.transform(titanic_df)

##### 3.9.2. features column이 추가된 것을 확인할 수 있습니다.

In [ ]:
feature_vector.show()

### 4. Modelling

#### 4.1. 모델 평가를 위해 train 데이터와 test 데이터를 약 80:20 비율로 준비합니다.

In [ ]:
(trainingData, testData) = feature_vector.randomSplit([0.8, 0.2], seed=11)

#### 4.2. [LogisticRegression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)
##### 분류에 사용하는 모델로 선형 함수 결과를 시그모이드 함수를 이용하여 0 ~ 1 사이로 압축합니다. 이진 분류는 0.5보다 높을 때는 True, 그이하는 Flase로 하여 모델을 학습시킵니다.

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol='survived', featuresCol='features')
pipeline = Pipeline(stages=[lr])
lrModel = pipeline.fit(trainingData)
lr_prediction = lrModel.transform(testData)
lr_prediction.select('prediction', 'survived', 'features').show()
evaluator = MulticlassClassificationEvaluator(labelCol='survived', predictionCol='prediction', metricName='accuracy')

#### Evaluating accuracy of LogisticRegression.

In [ ]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print("Accuracy of LogisticRegression is = %g" % (lr_accuracy))
print("Test Error of LogisticRegression = %g " % (1.0 - lr_accuracy))

#### 4.3. [DecisionTreeClassifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier)
##### 의사결정 규칙(rule)을 트리구조로 도표화하여 분류(classification)와 예측(prediction)을 수행하는 분석방법입니다.

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol='survived', featuresCol='features')
pipeline = Pipeline(stages=[dt])
dt_model = pipeline.fit(trainingData)
dt_prediction = dt_model.transform(testData)
dt_prediction.select('prediction', 'survived', 'features').show()

#### Evaluating accuracy of DecisionTreeClassifier.

In [ ]:
dt_accuracy = evaluator.evaluate(dt_prediction)
print("Accuracy of DecisionTreeClassifier is = %g" % (dt_accuracy))
print("Test Error of DecisionTreeClassifier = %g " % (1.0 - dt_accuracy))

#### 4.4. [RandomForestClassifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier)
##### 분류, 회귀분석 등에 사용되는 앙상블 학습 방법의 일종으로, 훈련 과정에서 구성한 다수의 결정 트리로 부터 분류 또는 평균 예측치(회귀분석)를 출력함으로써 동작합니다.

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = DecisionTreeClassifier(labelCol='survived', featuresCol='features')
pipeline = Pipeline(stages=[rf])
rf_model = pipeline.fit(trainingData)
rf_prediction = rf_model.transform(testData)
rf_prediction.select('prediction', 'survived', 'features').show()

#### Evaluating accuracy of RandomForestClassifier.

In [ ]:
rf_accuracy = evaluator.evaluate(rf_prediction)
print("Accuracy of RandomForestClassifier is = %g" % (rf_accuracy))
print("Test Error of RandomForestClassifier  = %g " % (1.0 - rf_accuracy))

#### 4.5. [Gradient-boosted tree classifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier)
##### Gradient-Boosted Trees (GBT)는 의사 결정 트리의 앙상블입니다. GBT는 손실 함수를 최소화하기 위해 의사 결정 트리를 반복적으로 훈련시킵니다.

In [ ]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol='survived', featuresCol='features', maxIter=10)
pipeline = Pipeline(stages=[gbt])
gbt_model = pipeline.fit(trainingData)
gbt_prediction = gbt_model.transform(testData)
gbt_prediction.select('prediction', 'survived', 'features').show()

#### Evaluate accuracy of Gradient-boosted.

In [ ]:
gbt_accuracy = evaluator.evaluate(gbt_prediction)
print("Accuracy of Gradient-boosted tree classifie is = %g" % (gbt_accuracy))
print("Test Error of Gradient-boosted tree classifie %g" % (1.0 - gbt_accuracy))

#### 4.6. [NaiveBayes](https://spark.apache.org/docs/latest/ml-classification-regression.html#naive-bayes)
##### 각 사건 특성들이 독립이라는 가정 하에 베이즈 정리(Bayes’ theorem)을 적용한 간단한 확률 기반 분류입니다.

In [ ]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(labelCol='survived', featuresCol='features')
pipeline = Pipeline(stages=[nb])
nb_model = pipeline.fit(trainingData)
nb_prediction = nb_model.transform(testData)
nb_prediction.select('prediction', 'survived', 'features').show()

#### Evaluating accuracy of NaiveBayes.

In [ ]:
nb_accuracy = evaluator.evaluate(nb_prediction)
print("Accuracy of NaiveBayes is  = %g" % (nb_accuracy))
print("Test Error of NaiveBayes  = %g " % (1.0 - nb_accuracy))

#### 4.7. [Support Vector Machine](https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-support-vector-machine)
##### 바이너리 분류 알고리즘으로 N차원 장소에서 2가지 유형의 점 집합이 주어지면 SVM은 (N-1) 차원의 초평면(hyperplane)을 생성하여, 이 점들을 두 그룹으로 분리합니다.

In [ ]:
from pyspark.ml.classification import LinearSVC

svm = LinearSVC(labelCol='survived', featuresCol='features')
pipeline = Pipeline(stages=[svm])
svm_model = pipeline.fit(trainingData)
svm_prediction = svm_model.transform(testData)
svm_prediction.select('prediction', 'survived', 'features').show()

#### Evaluating the accuracy of Support Vector Machine.

In [ ]:
svm_accuracy = evaluator.evaluate(svm_prediction)
print("Accuracy of Support Vector Machine is = %g" % (svm_accuracy))
print("Test Error of Support Vector Machine = %g " % (1.0 - svm_accuracy))

#### 4.8. Result

##### 위 테스트에서는 LogisticRegression과 Support Vector Machine이 83.6257%의 정확도로 가장 좋은 성능을 보여주었습니다.
##### 하지만, 아직 기본적인 테스트만 수행하였으므로 개선할 여지가 많이 남아있습니다.

### 5. Create your own model
![kaggle_leaderboard](images/kaggle_leaderboard.png)
#### 지금까지 다뤘던 기본적인 Feature engineering과 Modeling만으로 83.6%의 비교적 높은 정확도를 얻을 수 있었습니다. 
#### kaggle titanic competition에서 test 데이터에 대해서도 같은 정확도를 유지한다면 전체 10000명 중 상위 229등에 해당하는 높은 수치입니다. 하지만, 아직 개선할 여지가 많이 남아있습니다.
#### * 첫 번째 접근할 수 있는 방법으로는 새로운 feacure를 추가하거나 기존 feature를 모델에서 제외하는 것입니다.
#### * 두 번째 방법은 ML algorithm을 튜닝하는 것으로 아래 페이지를 참고하시면 됩니다.
(https://spark.apache.org/docs/latest/ml-tuning.html) 